In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("SparkDF USer Defined Functions").getOrCreate()

earthquakeDF= spark.read.option("header","true")\
                        .option("delimiter","\t")\
                        .option("inferSchema","True")\
                        .csv("datasets/EarthquakeDataset.txt")

earthquakeDF = earthquakeDF.select("Date","Latitude","Longitude","xM","Location")
#earthquakeDF.show()

def assingRegion(lat,lng):
    region = ""
    if lat >= 39:
        if lng <= 31:
            region ="Region 1"
        elif lng <= 36:
            region ="Region 2"        
        elif lng <= 41:
            region ="Region 3"
        else:
            region ="Region 4"
    else:
        if lng <= 31:
            region ="Region 5"
        elif lng <= 36:
            region ="Region 6"        
        elif lng <= 41:
            region ="Region 7"
        else:
            region ="Region 8"         
    return  region

assingUDF = UserDefinedFunction(assingRegion,StringType())
earthquakeDF  = earthquakeDF.withColumn("Region",assingUDF("Latitude","Longitude"))
earthquakeDF.show()
regionDF = earthquakeDF.groupBy("Region").count()
regionDF.createOrReplaceTempView("region")
#spark.sql("select Region, count from region order by count desc").show()
regionDF.sort("count",ascending=False).show()

+----------+--------+---------+---+--------------------+--------+
|      Date|Latitude|Longitude| xM|            Location|  Region|
+----------+--------+---------+---+--------------------+--------+
|2020.12.31| 37.8435|  26.7775|3.8|          EGE DENIZI|Region 5|
|2020.12.30| 36.4918|  28.7092|3.9|             AKDENIZ|Region 5|
|2020.12.30|  37.796|  26.4165|4.4|ONIKI ADALAR (AKD...|Region 5|
|2020.12.27| 38.4172|  39.1482|3.9|KAVAKKOY-SIVRICE ...|Region 7|
|2020.12.27|  36.475|  28.7852|4.2|             AKDENIZ|Region 5|
|2020.12.27|  38.505|   39.218|5.6|KAVAKTEPE- (ELAZI...|Region 7|
|2020.12.26| 38.5305|  39.2102|4.1|KAVAKTEPE- (ELAZI...|Region 7|
|2020.12.26| 38.0033|  42.9642|3.9|KORULU-CATAK (VAN...|Region 8|
|2020.12.23| 36.5213|   28.695|3.7|             AKDENIZ|Region 5|
|2020.12.23| 37.8888|  27.6075|3.7|DAGKARAAGAC-GERME...|Region 5|
|2020.12.22| 35.7737|   36.447|3.9|              SURIYE|Region 7|
|2020.12.20| 37.8967|  43.8062|4.5|AYDEMIR-BASKALE (...|Region 8|
|2020.12.2